# ARC Tools
### Visuallize OneDMin r_min and r_max
The r (min and max) distance is defined between the center of mass of the species and the center of ass of the bath gas.

#### input parameters:

In [ ]:
label = 'species_label'

xyz = """
C       0.05984800   -0.62319600    0.00000000
H      -0.46898100   -1.02444400    0.87886100
H      -0.46898100   -1.02444400   -0.87886100
H       1.08093800   -1.00826200    0.00000000
N       0.05980600    0.81236000    0.00000000
H      -0.92102100    1.10943400    0.00000000
"""

bath_gas = 'O2'  # allowed values are 'He', 'Ne', 'Ar', 'Kr', 'H2', 'N2', or 'O2'

r_min = 3
r_max = 6

In [ ]:
from arc.species import ARCSpecies
from arc.common import get_atom_radius
from arc.species.converter import get_center_of_mass, get_most_common_isotope_for_element, xyz_from_data
from arc.plotter import draw_3d


def plot_collider(spc, r, bath_gas):
    """
    Plot a species and its collider.

    Args:
        spc (ARCSpecies): The species with xyz data.
        r (float): The bath gas distance from the species' center of mass.
        bath_gas (str): The collider bath gas.
    """
    xyz = spc.get_xyz()
    cm = get_center_of_mass(xyz)
    
    if '2' in bath_gas:
        symbol = bath_gas[0]
        bath_gas_bond_length = {'H2': 0.743, 'N2': 1.1, 'O2': 1.208}  # Angstrom
        xyz = add_bath_gas_to_xyz(xyz, symbol, cm, r - 0.5 * bath_gas_bond_length[bath_gas])
        xyz = add_bath_gas_to_xyz(xyz, symbol, cm, r + 0.5 * bath_gas_bond_length[bath_gas])
    else:
        xyz = add_bath_gas_to_xyz(xyz, bath_gas, cm, r)
    draw_3d(xyz=xyz, species=spc)


def add_bath_gas_to_xyz(xyz, symbol, cm, r):
    """
    Add an atom (bath gas) at distance r from the center of mass to xyz and return xyz.

    Args:
        xyz (dict): The coordinates to process.
        symbol (str): The bath gas atom symbol.
        cm (tuple): The center of mass coordinates.
        r (float): The bath gas distance from the species' center of mass.

    Returns:
        dict: The xyz coordinates.
    """
    symbols = xyz['symbols'] + (symbol, ) * 6
    isotopes = xyz['isotopes'] + (get_most_common_isotope_for_element(symbol), ) * 6
    coords = [coord for coord in xyz['coords']] \
        + [(cm[0] + r, cm[1], cm[2])] \
        + [(cm[0], cm[1] + r, cm[2])] \
        + [(cm[0], cm[1], cm[2] + r)] \
        + [(cm[0] - r, cm[1], cm[2])] \
        + [(cm[0], cm[1] - r, cm[2])] \
        + [(cm[0], cm[1], cm[2] - r)]
    xyz = xyz_from_data(coords=coords, symbols=symbols, isotopes=isotopes)
    return xyz


spc = ARCSpecies(label=label, xyz=xyz)

print(f'Species {label} has a radius of {spc.radius:.2f} Angstrom')

print(f'\n\nr_min: {r_min} Angstrom')
plot_collider(spc, r_min, bath_gas)
print(f'\n\nr_max: {r_max} Angstrom')
plot_collider(spc, r_max, bath_gas)